In [18]:
import numpy as np
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from keras.callbacks import EarlyStopping, LambdaCallback
from termcolor import colored

In [19]:
all_chars='0123456789+'
num_features = len(all_chars)
print('no of features:', num_features)
char_to_index= dict((c,i) for i,c in enumerate(all_chars))
index_to_char= dict((i,c) for i, c in enumerate(all_chars))

no of features: 11


In [20]:
def generate_data():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    example = str(first)+ '+' + str(second)
    label = str(first+second)
    return example, label
generate_data()

('29+27', '56')

In [21]:
hidden_units=128
max_time_steps=5
model = Sequential([
    SimpleRNN(hidden_units,input_shape=(None,num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units,return_sequences=True),
    TimeDistributed(Dense(num_features,activation='softmax'))
]
)
model.compile(
   loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 128)               17920     
                                                                 
 repeat_vector_2 (RepeatVect  (None, 5, 128)           0         
 or)                                                             
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 5, 128)            32896     
                                                                 
 time_distributed_2 (TimeDis  (None, 5, 11)            1419      
 tributed)                                                       
                                                                 
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [22]:
def vectorize_example(example,label):
    x=np.zeros((max_time_steps,num_features))
    y=np.zeros((max_time_steps,num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i,c in enumerate(example):
        x[i+diff_x,char_to_index[c]] =1
    for i in range(diff_x):
        x[i,char_to_index['0']] = 1
    for i,c in enumerate(label):
        y[i+diff_y,char_to_index[c]] =1
    for i in range(diff_y):
        y[i,char_to_index['0']] = 1  
    return x,y
e, l = generate_data()
print(e,l)
x,y= vectorize_example(e,l)
print(x.shape,y.shape)

35+50 85
(5, 11) (5, 11)


In [23]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i,vec in enumerate(example)]
    return ''.join(result)
devectorize_example(x)

'35+50'

In [24]:
devectorize_example(y)

'00085'

In [25]:
def create_dataset(num_examples=2000):
    x=np.zeros((num_examples,max_time_steps,num_features))
    y=np.zeros((num_examples,max_time_steps,num_features))
    for i in range(num_examples):
        e,l = generate_data()
        e_v, l_v = vectorize_example(e,l)
        x[i] = e_v
        y[i] = l_v
    return x,y
x,y = create_dataset()
print(x.shape,y.shape)

(2000, 5, 11) (2000, 5, 11)


In [26]:
devectorize_example(x[0])
devectorize_example(y[0])

'00094'

In [36]:
l_cb=LambdaCallback(
    on_epoch_end = lambda e,l: print('{:.2f}'.format(l['val_accuracy']),end=' _ ')
)
es_cb=EarlyStopping(monitor='val_loss',patience=10)
model.fit(x,y,epochs=500,batch_size=256,validation_split=0.2,
         verbose=False,callbacks=[es_cb,l_cb])

0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 

In [37]:
x_test,y_test = create_dataset(10)
preds = model.predict(x_test)
for i,pred in enumerate(preds):
    y=devectorize_example(y_test[i])
    y_hat = devectorize_example(pred)
    col='green'
    if y!= y_hat:
        col='red'
    out='Input: '+ devectorize_example(x_test[i])+' Out: ' +y+'Pred:' +y_hat
    print(colored(out,col))

1/1 [==============================] - 1s 519ms/step
Input: 33+92 Out: 00125Pred:00125
Input: 79+80 Out: 00159Pred:00159
Input: 96+73 Out: 00169Pred:00169
Input: 50+30 Out: 00080Pred:00080
Input: 26+47 Out: 00073Pred:00073
Input: 34+69 Out: 00103Pred:00104
Input: 91+52 Out: 00143Pred:00143
Input: 34+27 Out: 00061Pred:00061
Input: 77+60 Out: 00137Pred:00137
Input: 13+96 Out: 00109Pred:00119
